**topic identification**


find in this notebook examples to following metacharacter setting can be found:
- [Anchors](#anchors)
- [Quantifiers](#quantifiers)
- [Disjunctions](#disjunctions)
- [Character Classes](#character_classes)

python documentation:
- Regular Expression Syntax [library "re"](https://docs.python.org/3/library/re.html)
- How-to use re [re-how-to](https://docs.python.org/3/howto/regex.html#regex-howto)



In [2]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
# path to dtaset-file
#
import os
notebook_path = os.path.abspath("nlp_2_1_topic_identification.ipynb")
#
# data_file = os.path.join(os.path.dirname(notebook_path), "NLP\data\subset_NewsCategory_v2.json")
data_file = os.path.join(os.path.dirname(notebook_path), "NLP\data\\News_Category_Dataset_v2.json")

#
#data_file = "D://Python_382//NLP//data//News_Category_Dataset_v2.json"
# df = pd.read_json(data_file, orient="records", lines=True)
#
fJson = open(data_file)
fJson.close()

In [5]:
# read data from json
df = pd.read_json(data_file, orient="records", lines=True)
#
data = pd.DataFrame()
data["text"] = df.headline + df.short_description
data["labels"] = df.category
#
labels = list(data["labels"].unique())
#
# Convert labels in numeric values
le = preprocessing.LabelEncoder()
le.fit(labels)
data["labels"] = le.transform(data["labels"])
#
train_df, eval_df = train_test_split(data, test_size=0.2)
#

In [4]:
# create a classification model
# model = ClassificationModel('bert', 'bert-base-uncased', num_labels=len(labels), use_cuda=False)
model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# train the model
# model.train_model(train_df, output_dir = os.path.join(os.path.dirname(notebook_path), "NLP/output/"))
model.train_model(train_df)

In [ ]:
# evaluate the model
result, model_outputs, predictions = model.eval_model(eval_def) 

In [12]:
from transformers import BertTokenizer, BertModel
#
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)